In [1]:
# ============================================================
# CELL 1: Load Embeddings and Prepare for FAISS
# ============================================================
import numpy as np
import faiss
import time
import pandas as pd

# Paths
EMBED_DIR = '../data/embeddings'
DATA_DIR = '../data/processed'
IMAGE_DIR = '../data/raw/images'

# Load fine-tuned embeddings (these are our best ones)
embeddings = np.load(f'{EMBED_DIR}/finetuned_embeddings.npy')
ids = np.load(f'{EMBED_DIR}/finetuned_ids.npy')

# Load metadata for displaying results
df = pd.read_csv(f'{DATA_DIR}/filtered_styles.csv')

print(f"Embeddings shape: {embeddings.shape}")
print(f"Data type: {embeddings.dtype}")
print(f"Memory: {embeddings.nbytes / 1e6:.1f} MB")

# --- STEP 1: Convert to float32 ---
# FAISS requires float32 (not float64)
embeddings = embeddings.astype(np.float32)
print(f"\nAfter float32 conversion:")
print(f"  Data type: {embeddings.dtype}")
print(f"  Memory: {embeddings.nbytes / 1e6:.1f} MB")

# --- STEP 2: L2 Normalize ---
# This makes L2 distance equivalent to cosine similarity
faiss.normalize_L2(embeddings)

# Verify normalization: each vector should have length = 1.0
norms = np.linalg.norm(embeddings, axis=1)
print(f"\nAfter L2 normalization:")
print(f"  Min norm: {norms.min():.6f}")
print(f"  Max norm: {norms.max():.6f}")
print(f"  (Should both be ~1.0)")

# --- STEP 3: Key dimensions ---
n_vectors, dimension = embeddings.shape
print(f"\nReady for FAISS:")
print(f"  {n_vectors:,} vectors")
print(f"  {dimension} dimensions each")
print(f"  {embeddings.nbytes / 1e6:.1f} MB total")


Embeddings shape: (43916, 2048)
Data type: float32
Memory: 359.8 MB

After float32 conversion:
  Data type: float32
  Memory: 359.8 MB

After L2 normalization:
  Min norm: 1.000000
  Max norm: 1.000000
  (Should both be ~1.0)

Ready for FAISS:
  43,916 vectors
  2048 dimensions each
  359.8 MB total


In [2]:
# ============================================================
# CELL 2: Flat Index — Exact Nearest Neighbor Search
# ============================================================
import faiss
import numpy as np
import time

# --- STEP 1: Build the index ---
print("Building Flat Index (exact search)...")
start = time.time()

flat_index = faiss.IndexFlatL2(dimension)  # dimension = 2048
flat_index.add(embeddings)                 # add all vectors

build_time = time.time() - start
print(f"  Build time: {build_time*1000:.1f} ms")
print(f"  Vectors in index: {flat_index.ntotal:,}")

# --- STEP 2: Test single query ---
# Pick a random image as query
query_id = ids[0]
query_vector = embeddings[0].reshape(1, -1)  # FAISS needs shape (1, 2048)

start = time.time()
distances, indices = flat_index.search(query_vector, k=5)
search_time = time.time() - start

print(f"\n  Single query time: {search_time*1000:.2f} ms")

# Show results
query_type = df[df['id'] == query_id]['articleType'].values[0]
print(f"\nQuery: ID={query_id}, Type={query_type}")
print(f"{'Rank':<6} {'ID':<10} {'Type':<20} {'L2 Distance':<12} {'Cosine Sim':<10}")
print("-" * 60)

for rank in range(5):
    result_idx = indices[0][rank]
    result_id = ids[result_idx]
    result_type = df[df['id'] == result_id]['articleType'].values[0]
    l2_dist = distances[0][rank]
    cos_sim = 1 - l2_dist / 2  # convert L2² back to cosine similarity
    match = "✓" if result_type == query_type else "✗"
    print(f"  {rank+1:<4} {result_id:<10} {result_type:<20} {l2_dist:<12.4f} {cos_sim:<10.4f} {match}")

# --- STEP 3: Benchmark — run 100 queries ---
print(f"\nBenchmarking 100 queries...")
query_vectors = embeddings[:100]  # first 100 as queries

start = time.time()
distances, indices = flat_index.search(query_vectors, k=5)
total_time = time.time() - start

print(f"  Total time for 100 queries: {total_time*1000:.1f} ms")
print(f"  Average per query: {total_time*10:.2f} ms")
print(f"  Queries per second: {100/total_time:.0f}")



Building Flat Index (exact search)...
  Build time: 20.7 ms
  Vectors in index: 43,916

  Single query time: 7.38 ms

Query: ID=15970, Type=Shirts
Rank   ID         Type                 L2 Distance  Cosine Sim
------------------------------------------------------------
  1    15970      Shirts               0.0000       1.0000     ✓
  2    31627      Shirts               0.0060       0.9970     ✓
  3    8741       Shirts               0.0063       0.9968     ✓
  4    9646       Shirts               0.0063       0.9968     ✓
  5    28806      Shirts               0.0073       0.9963     ✓

Benchmarking 100 queries...
  Total time for 100 queries: 47.2 ms
  Average per query: 0.47 ms
  Queries per second: 2117


In [3]:
# ============================================================
# CELL 3: IVF Index — Clustered Nearest Neighbor Search
# ============================================================
import faiss
import numpy as np
import time

# --- STEP 1: Create IVF Index ---
nlist = 100  # number of clusters

# IVF needs a "quantizer" — the index used to find nearest cluster
quantizer = faiss.IndexFlatL2(dimension)  # exact search over 100 centroids

# Create the IVF index
ivf_index = faiss.IndexIVFFlat(quantizer, dimension, nlist)

# --- STEP 2: Train the index (run K-means) ---
print("Training IVF Index (running K-means clustering)...")
start = time.time()
ivf_index.train(embeddings)  # learns the 100 cluster centroids
train_time = time.time() - start
print(f"  Train time: {train_time:.2f} seconds")

# --- STEP 3: Add vectors to the index ---
start = time.time()
ivf_index.add(embeddings)
add_time = time.time() - start
print(f"  Add time: {add_time*1000:.1f} ms")
print(f"  Vectors in index: {ivf_index.ntotal:,}")

# --- STEP 4: Test with different nprobe values ---
query_vectors = embeddings[:100]  # 100 test queries

# Get ground truth from Flat Index
_, true_neighbors = flat_index.search(query_vectors, k=5)

print(f"\nBenchmarking IVF with different nprobe values:")
print(f"{'nprobe':<10} {'Time (ms)':<12} {'Queries/sec':<14} {'Recall@5':<10}")
print("-" * 48)

for nprobe in [1, 2, 5, 10, 20, 50, 100]:
    ivf_index.nprobe = nprobe
    
    # Speed test
    start = time.time()
    distances, indices = ivf_index.search(query_vectors, k=5)
    elapsed = time.time() - start
    
    # Accuracy test: how many true neighbors did we find?
    recall = 0
    for i in range(100):
        true_set = set(true_neighbors[i])
        found_set = set(indices[i])
        recall += len(true_set & found_set) / 5
    recall /= 100  # average over 100 queries
    
    qps = 100 / elapsed
    print(f"{nprobe:<10} {elapsed*1000:<12.1f} {qps:<14.0f} {recall:<10.2%}")


Training IVF Index (running K-means clustering)...
  Train time: 0.37 seconds
  Add time: 79.4 ms
  Vectors in index: 43,916

Benchmarking IVF with different nprobe values:
nprobe     Time (ms)    Queries/sec    Recall@5  
------------------------------------------------
1          7.9          12588          77.20%    
2          14.2         7031           92.80%    
5          31.8         3144           99.00%    
10         63.4         1578           100.00%   
20         122.7        815            100.00%   
50         319.8        313            100.00%   
100        530.6        188            100.00%   


In [4]:
# ============================================================
# CELL 4: IVF + PQ Index — Compressed Nearest Neighbor Search
# ============================================================
import faiss
import numpy as np
import time

# --- Parameters ---
nlist = 100   # number of clusters (same as IVF)
m = 64        # number of sub-vectors (2048 / 64 = 32 dims each)
nbits = 8     # bits per code (8 bits = 256 possible codes per sub-vector)

# --- STEP 1: Create IVF+PQ Index ---
print("Building IVF+PQ Index (clustered + compressed)...")
quantizer = faiss.IndexFlatL2(dimension)
ivfpq_index = faiss.IndexIVFPQ(quantizer, dimension, nlist, m, nbits)

# --- STEP 2: Train (learns clusters AND codebooks) ---
start = time.time()
ivfpq_index.train(embeddings)
train_time = time.time() - start
print(f"  Train time: {train_time:.2f} seconds")

# --- STEP 3: Add vectors ---
start = time.time()
ivfpq_index.add(embeddings)
add_time = time.time() - start
print(f"  Add time: {add_time*1000:.1f} ms")

# --- STEP 4: Compare memory usage ---
flat_memory = embeddings.nbytes
ivfpq_memory = ivfpq_index.ntotal * m  # each vector = m bytes

print(f"\n  Memory comparison:")
print(f"    Flat Index:   {flat_memory / 1e6:.1f} MB")
print(f"    IVF+PQ Index: {ivfpq_memory / 1e6:.1f} MB")
print(f"    Compression:  {flat_memory / ivfpq_memory:.0f}x smaller!")

# --- STEP 5: Benchmark with different nprobe values ---
query_vectors = embeddings[:100]
_, true_neighbors = flat_index.search(query_vectors, k=5)

print(f"\nBenchmarking IVF+PQ with different nprobe values:")
print(f"{'nprobe':<10} {'Time (ms)':<12} {'Queries/sec':<14} {'Recall@5':<10}")
print("-" * 48)

for nprobe in [1, 2, 5, 10, 20, 50, 100]:
    ivfpq_index.nprobe = nprobe
    
    start = time.time()
    distances, indices = ivfpq_index.search(query_vectors, k=5)
    elapsed = time.time() - start
    
    recall = 0
    for i in range(100):
        true_set = set(true_neighbors[i])
        found_set = set(indices[i])
        recall += len(true_set & found_set) / 5
    recall /= 100
    
    qps = 100 / elapsed
    print(f"{nprobe:<10} {elapsed*1000:<12.1f} {qps:<14.0f} {recall:<10.2%}")


Building IVF+PQ Index (clustered + compressed)...
  Train time: 15.36 seconds
  Add time: 956.6 ms

  Memory comparison:
    Flat Index:   359.8 MB
    IVF+PQ Index: 2.8 MB
    Compression:  128x smaller!

Benchmarking IVF+PQ with different nprobe values:
nprobe     Time (ms)    Queries/sec    Recall@5  
------------------------------------------------
1          1.9          53911          56.60%    
2          2.1          46927          62.00%    
5          2.6          38607          62.60%    
10         3.6          27979          62.60%    
20         5.6          17706          62.60%    
50         12.4         8094           62.60%    
100        20.4         4903           62.60%    


In [5]:
# ============================================================
# CELL 4b: IVF+PQ with More Sub-vectors (faster than OPQ)
# ============================================================
import faiss
import numpy as np
import time

# --- Use more sub-vectors for better accuracy ---
nlist = 100
m = 128       # 128 sub-vectors instead of 64 (each piece = 16 dims)
nbits = 8     # 8 bits per code

print("Building IVF+PQ Index (m=128, better accuracy)...")
quantizer = faiss.IndexFlatL2(dimension)
ivfpq2_index = faiss.IndexIVFPQ(quantizer, dimension, nlist, m, nbits)

# Train
start = time.time()
ivfpq2_index.train(embeddings)
train_time = time.time() - start
print(f"  Train time: {train_time:.2f} seconds")

# Add vectors
start = time.time()
ivfpq2_index.add(embeddings)
add_time = time.time() - start
print(f"  Add time: {add_time*1000:.1f} ms")

# Memory
pq2_memory = ivfpq2_index.ntotal * m
print(f"\n  Memory: {pq2_memory / 1e6:.1f} MB")
print(f"  Compression: {embeddings.nbytes / pq2_memory:.0f}x smaller than Flat")

# Benchmark
query_vectors = embeddings[:100]
_, true_neighbors = flat_index.search(query_vectors, k=5)

print(f"\nBenchmarking IVF+PQ (m=128):")
print(f"{'nprobe':<10} {'Time (ms)':<12} {'Queries/sec':<14} {'Recall@5':<10}")
print("-" * 48)

for nprobe in [1, 2, 5, 10, 20, 50, 100]:
    ivfpq2_index.nprobe = nprobe
    
    start = time.time()
    distances, indices = ivfpq2_index.search(query_vectors, k=5)
    elapsed = time.time() - start
    
    recall = 0
    for i in range(100):
        true_set = set(true_neighbors[i])
        found_set = set(indices[i])
        recall += len(true_set & found_set) / 5
    recall /= 100
    
    qps = 100 / elapsed
    print(f"{nprobe:<10} {elapsed*1000:<12.1f} {qps:<14.0f} {recall:<10.2%}")


Building IVF+PQ Index (m=128, better accuracy)...
  Train time: 29.46 seconds
  Add time: 2830.0 ms

  Memory: 5.6 MB
  Compression: 64x smaller than Flat

Benchmarking IVF+PQ (m=128):
nprobe     Time (ms)    Queries/sec    Recall@5  
------------------------------------------------
1          3.0          32799          62.40%    
2          3.4          29335          68.20%    
5          4.7          21441          70.20%    
10         6.8          14661          70.60%    
20         12.0         8351           70.60%    
50         31.4         3187           70.60%    
100        46.9         2132           70.60%    


In [6]:
# ============================================================
# CELL 5: Final Comparison — All Index Types
# ============================================================
import numpy as np
import time

query_vectors = embeddings[:100]
_, true_neighbors = flat_index.search(query_vectors, k=5)

print("=" * 75)
print("FINAL COMPARISON: All FAISS Index Types")
print("=" * 75)
print(f"{'Index':<20} {'Memory':<12} {'nprobe':<8} {'Speed (q/s)':<14} {'Recall@5':<10}")
print("-" * 75)

# 1. Flat Index
start = time.time()
flat_index.search(query_vectors, k=5)
elapsed = time.time() - start
print(f"{'Flat (exact)':<20} {'359.8 MB':<12} {'n/a':<8} {100/elapsed:<14.0f} {'100.00%':<10}")

# 2. IVF (nprobe=5)
ivf_index.nprobe = 5
start = time.time()
d, idx = ivf_index.search(query_vectors, k=5)
elapsed = time.time() - start
recall = sum(len(set(true_neighbors[i]) & set(idx[i]))/5 for i in range(100)) / 100
print(f"{'IVF (nprobe=5)':<20} {'359.8 MB':<12} {'5':<8} {100/elapsed:<14.0f} {recall:<10.2%}")

# 3. IVF (nprobe=10)
ivf_index.nprobe = 10
start = time.time()
d, idx = ivf_index.search(query_vectors, k=5)
elapsed = time.time() - start
recall = sum(len(set(true_neighbors[i]) & set(idx[i]))/5 for i in range(100)) / 100
print(f"{'IVF (nprobe=10)':<20} {'359.8 MB':<12} {'10':<8} {100/elapsed:<14.0f} {recall:<10.2%}")

# 4. IVF+PQ (m=64)
ivfpq_index.nprobe = 10
start = time.time()
d, idx = ivfpq_index.search(query_vectors, k=5)
elapsed = time.time() - start
recall = sum(len(set(true_neighbors[i]) & set(idx[i]))/5 for i in range(100)) / 100
print(f"{'IVF+PQ (m=64)':<20} {'2.8 MB':<12} {'10':<8} {100/elapsed:<14.0f} {recall:<10.2%}")

# 5. IVF+PQ (m=128)
ivfpq2_index.nprobe = 10
start = time.time()
d, idx = ivfpq2_index.search(query_vectors, k=5)
elapsed = time.time() - start
recall = sum(len(set(true_neighbors[i]) & set(idx[i]))/5 for i in range(100)) / 100
print(f"{'IVF+PQ (m=128)':<20} {'5.6 MB':<12} {'10':<8} {100/elapsed:<14.0f} {recall:<10.2%}")

print("-" * 75)

print("""
RECOMMENDATION:
  For our dataset (44K vectors, 360 MB):
    → Use IVF with nprobe=10 (100% recall, fast enough)
    → Flat Index also fine (exact results, simple to maintain)

  For production at scale (1M+ vectors):
    → Use IVF+PQ for memory savings
    → Accept ~70% recall trade-off OR use re-ranking to recover accuracy

  For this project:
    → We'll use IVF (nprobe=10) as our production index
""")


FINAL COMPARISON: All FAISS Index Types
Index                Memory       nprobe   Speed (q/s)    Recall@5  
---------------------------------------------------------------------------
Flat (exact)         359.8 MB     n/a      2183           100.00%   
IVF (nprobe=5)       359.8 MB     5        1734           99.00%    
IVF (nprobe=10)      359.8 MB     10       1581           100.00%   
IVF+PQ (m=64)        2.8 MB       10       15967          62.60%    
IVF+PQ (m=128)       5.6 MB       10       11869          70.60%    
---------------------------------------------------------------------------

RECOMMENDATION:
  For our dataset (44K vectors, 360 MB):
    → Use IVF with nprobe=10 (100% recall, fast enough)
    → Flat Index also fine (exact results, simple to maintain)

  For production at scale (1M+ vectors):
    → Use IVF+PQ for memory savings
    → Accept ~70% recall trade-off OR use re-ranking to recover accuracy

  For this project:
    → We'll use IVF (nprobe=10) as our produc

In [7]:
# ============================================================
# CELL 6: Save FAISS Indexes to Disk
# ============================================================
import faiss
import os

INDEX_DIR = '../indexing'
os.makedirs(INDEX_DIR, exist_ok=True)

# Save Flat Index
faiss.write_index(flat_index, f'{INDEX_DIR}/flat_index.faiss')
flat_size = os.path.getsize(f'{INDEX_DIR}/flat_index.faiss')
print(f"Saved: flat_index.faiss ({flat_size/1e6:.1f} MB)")

# Save IVF Index
faiss.write_index(ivf_index, f'{INDEX_DIR}/ivf_index.faiss')
ivf_size = os.path.getsize(f'{INDEX_DIR}/ivf_index.faiss')
print(f"Saved: ivf_index.faiss ({ivf_size/1e6:.1f} MB)")

# Verify by loading back
test_index = faiss.read_index(f'{INDEX_DIR}/flat_index.faiss')
print(f"\nVerification:")
print(f"  Loaded flat_index: {test_index.ntotal:,} vectors")

# Test search on loaded index
query = embeddings[0].reshape(1, -1)
distances, indices = test_index.search(query, k=3)
print(f"  Test search: top 3 IDs = {[ids[i] for i in indices[0]]}")
print(f"  Distances: {distances[0]}")
print(f"\n✅ Indexes saved and verified!")


Saved: flat_index.faiss (359.8 MB)
Saved: ivf_index.faiss (360.9 MB)

Verification:
  Loaded flat_index: 43,916 vectors
  Test search: top 3 IDs = [np.int64(15970), np.int64(31627), np.int64(8741)]
  Distances: [0.         0.00600655 0.00633012]

✅ Indexes saved and verified!
